In [1]:
import sys
sys.path.append('d:/flagellar/code/core')
sys.path.append('/flagellar/code/core/')
sys.path.append('/kaggle/input/my-flagellar-library/')
from typing import Tuple
import os
import glob
import shutil
import scipy

import zarr
from cryoet_data_portal import Client, Dataset
import numpy as np
import pandas as pd

import sys
sys.path.append('d:/flagellar/code/core')
sys.path.append('/kaggle/code/core')
sys.path.append('/kaggle/input/my-flagellar-library/')
import flg_support as fls
import copy
import matplotlib.pyplot as plt
import h5py
import cv2
import flg_numerics

MainProcess


In [2]:
%%writefile irregular_labels.csv

tomo_id,z
aba2014-02-21-14,18.0
mba2011-02-16-103,62.0
mba2011-02-16-106,66.0
mba2011-02-16-108,42.0
mba2011-02-16-111,56.0
mba2011-02-16-115,70.0
mba2011-02-16-116,68.0
mba2011-02-16-11,66.0
mba2011-02-16-122,42.0
mba2011-02-16-123,62.0
mba2011-02-16-129,64.0
mba2011-02-16-12,60.0
mba2011-02-16-133,59.0
mba2011-02-16-139,60.0
mba2011-02-16-141,66.0
mba2011-02-16-143,57.0
mba2011-02-16-145,48.0
mba2011-02-16-145,51.0
mba2011-02-16-147,51.0
mba2011-02-16-150,62.0
mba2011-02-16-153,55.0
mba2011-02-16-153,63.0
mba2011-02-16-155,33.0
mba2011-02-16-157,60.0
mba2011-02-16-15,56.0
mba2011-02-16-15,50.0
mba2011-02-16-160,60.0
mba2011-02-16-160,51.0
mba2011-02-16-162,62.0
mba2011-02-16-170,62.0
mba2011-02-16-173,52.0
mba2011-02-16-176,59.0
mba2011-02-16-17,68.0
mba2011-02-16-19,70.0
mba2011-02-16-1,66.0
mba2011-02-16-1,46.0
mba2011-02-16-20,65.0
mba2011-02-16-23,59.0
mba2011-02-16-26,59.0
mba2011-02-16-27,63.0
mba2011-02-16-28,59.0
mba2011-02-16-28,68.0
mba2011-02-16-29,65.0
mba2011-02-16-30,66.0
mba2011-02-16-32,75.0
mba2011-02-16-33,56.0
mba2011-02-16-34,54.0
mba2011-02-16-35,57.0
mba2011-02-16-37,70.0
mba2011-02-16-3,63.0
mba2011-02-16-40,59.0
mba2011-02-16-40,55.0
mba2011-02-16-42,44.0
mba2011-02-16-42,26.0
mba2011-02-16-46,64.0
mba2011-02-16-48,63.0
mba2011-02-16-52,59.0
mba2011-02-16-53,54.0
mba2011-02-16-55,46.0
mba2011-02-16-60,54.0
mba2011-02-16-64,56.0
mba2011-02-16-65,66.0
mba2011-02-16-67,60.0
mba2011-02-16-68,60.0
mba2011-02-16-71,59.0
mba2011-02-16-75,47.0
mba2011-02-16-79,43.0
mba2011-02-16-79,44.0
mba2011-02-16-88,69.0
mba2011-02-16-90,60.0
mba2011-02-16-95,63.0

Overwriting irregular_labels.csv


In [3]:
import polars as pl
irregular_df = pl.read_csv("irregular_labels.csv")
irregular_tomo_ids = irregular_df["tomo_id"].unique(maintain_order=True).to_list()

In [4]:
# fls.remove_and_make_dir(fls.temp_dir + '/extra_data/')
# !kaggle datasets metadata -p /flagellar/temp/extra_data/ jeroencottaar/flagellar-motors-extra-data-preprocessed-2

In [5]:
client = Client()
extra_labels = pd.read_csv('/flagellar/data/labels.csv').rename(columns={"Motor axis 0": "z", "Motor axis 1": "y", "Motor axis 2": "x"})
ds_all = Dataset.find(client, [Dataset.authors.name == "Yi-Wei Chang"])+Dataset.find(client, [Dataset.authors.name == "Ariane Briegel"])+Dataset.find(client, [Dataset.authors.name == "Morgan Beeby"])

In [ ]:
import cupyx.scipy.signal
import cupyx.scipy.ndimage
import cupy as cp
target_voxel_spacing = 15.
#name_list = extra_labels['tomo_id']
c=0
nnn=[]
#os.makedirs('/kaggle/working/', exist_ok=True)
n_done = 0
range_slices = 8
for (i,d) in enumerate(ds_all):
    print(i, len(ds_all),c)
    for r in d.runs:
        labels = copy.deepcopy(extra_labels[extra_labels['tomo_id']==r.name]).reset_index()        
        if len(labels)==0:
            continue   
        if os.path.isfile(fls.temp_dir + '/extra_data/' + r.name + '/info.pickle'):
            continue
        fls.remove_and_make_dir(fls.temp_dir + '/extra_data/' + r.name)
        print(r.name)
        c+=1
        tt = r.tomograms[0]
        zar = zarr.open(tt.https_omezarr_dir, mode='r')[0]

        res = dict()      

        img = zar[...]
        if r.name in irregular_tomo_ids:
            print('irregular')
            img = img.astype(np.uint8).astype(np.float32)
        else:
            img  = img.astype(np.float32)
        img = cp.array(img)
 
        res['original_voxel_spacing'] = r.tomograms[0].tomogram_voxel_spacing.voxel_spacing
        resize_factor = res['original_voxel_spacing']/target_voxel_spacing

        new_shape = tuple(np.round(np.array(img.shape)*resize_factor).astype(int))
        
        for ii in range(img.shape[0]):
            img[ii,:,:] = flg_numerics.fourier_resample_nd(img[ii,:,:], (new_shape[1],new_shape[2]))

        for ii in range(img.shape[1]):
            for jj in range(img.shape[2]):
                
        
        os.makedirs('/kaggle/temp/volumes/' + r.name, exist_ok=True)
        for ii in range(img.shape[0]):
            this_img = img[ii,:,:]
            min_val = cp.percentile(this_img, 0.1)
            max_val = cp.percentile(this_img, 99.9)
            this_img = (this_img-min_val)/(max_val-min_val)
            this_img = cp.clip(this_img, 0, 1)
            this_img = (255*this_img).astype(cp.uint8)  
            cv2.imwrite(fls.temp_dir + '/extra_data/' + r.name + f'/slice_{ii:04d}.jpg', cp.asnumpy(this_img))

        labels['z'] = labels['z']/128*zar.shape[0]*resize_factor
        labels['y'] = labels['y']/512*zar.shape[1]*resize_factor
        labels['x'] = labels['x']/512*zar.shape[2]*resize_factor
        res['voxel_spacing'] = r.tomograms[0].tomogram_voxel_spacing.voxel_spacing/resize_factor
        res['labels'] = labels
        res['orig_size'] = zar.shape
        res['new_size'] = img.shape

        plt.figure()
        z = np.round(labels['z']).astype(int).item()
        plt.imshow(cv2.imread(fls.temp_dir + '/extra_data/' + r.name + f'/slice_{(z):04d}.jpg'))
        plt.scatter(labels['x'], labels['y'])
        plt.title(str(resize_factor))

        fls.dill_save(fls.temp_dir + '/extra_data/' + r.name + '/info.pickle', res)

        n_done += 1
        #if n_done%200==0:
        #    !kaggle datasets version -p  /kaggle/temp/volumes/ --dir-mode zip -m 'Update'

        print(c,n_done) 

In [ ]:
for ii in range(img.shape[0]):
    img[ii,:,:] = flg_numerics.fourier_resample_nd(img[ii,:,:], (new_shape[1],new_shape[2]))